In [2]:
# Vizualizing the data from storm sweeps

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
import pickle
from multiprocessing import Pool

import seaborn as sns

from weis.aeroelasticse import FileTools

sys.path.append(os.path.abspath("/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools"))

from fatigue import *

/var/folders/6r/0bmv_2bd2fx38vj0q38h019cv01fld/T/ipykernel_30372/234068428.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Locations of the folders
tc_steady = 'eagleData/23JAN24/steady/'
tc_unsteady = 'eagleData/23JAN24/unsteady/'

tc_fast_steady = 'eagleData/23JAN24-Faster/steady/'
tc_fast_unsteady = 'eagleData/23JAN24-Faster/unsteady/'

of351_steady = 'eagleData/23JAN24-of-351/steady/'
of351_unsteady = 'eagleData/23JAN24-of-351/unsteady/'

cm_dict = FileTools.load_yaml('eagleData/23JAN24/steady/case_matrix.yaml', package=1)

cm_df = pd.DataFrame(cm_dict)

In [ ]:
# listing the yaw angles
# cm_df.keys()
np.array(cm_df[cm_df.columns[10]]).astype(float).round(0).astype(int)

In [4]:
def process_Stats(openfastDF, fileName= None, startTime = 420.0):
    # Passed the OpenFAST dataframe, returns a dataframe with the stats for all the channels

    statsDF = pd.DataFrame(index = openfastDF.columns,  columns=['mean', 'std', 'max', 'min', 'DEL5', 'DEL10', 'DEL12'])

    success = True

    # Trim the data to start at startTime, if empty, write to error file and use whole data
    temp = openfastDF[openfastDF[('Time', '(s)')] >= startTime]

    if temp.empty:
        print(f'No data after {startTime} seconds')
        
        with open('error.txt', 'a') as f:
            f.write(f"{fileName} has no data after {openfastDF[('Time', '(s)')][len(openfastDF[('Time', '(s)')])-1]} seconds\n")

        temp = openfastDF

        success = False

    for col in temp.columns:

        # print(f'working on {col}')

        if col == ('Time', '(s)'):
            continue

        statsDF.loc[col] = [temp[col].mean(), temp[col].std(), temp[col].max(), temp[col].min(),
                            equivalent_load(temp[('Time', '(s)')], temp[col], m=5), equivalent_load(temp[('Time', '(s)')], temp[col], m=10), equivalent_load(temp[('Time', '(s)')], temp[col], m=12)]


    return statsDF, success

In [60]:
# Loading a single pickel file for testing
df = pd.read_pickle('eagleData/23JAN24/steady/storm-test_08.out.pkl')

# dump to csv with tab delimiter
df.to_csv('eagleData/23JAN24/steady/storm-test_08.out.csv', sep='\t', index=False)


In [ ]:
for file in os.listdir(tc_steady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(tc_steady + file)
        df.to_csv(tc_steady + file.replace('.pkl', '.csv'), sep='\t', index=False)

for file in os.listdir(tc_unsteady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(tc_unsteady + file)
        df.to_csv(tc_unsteady + file.replace('.pkl', '.csv'), sep='\t', index=False)

for file in os.listdir(tc_fast_steady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(tc_fast_steady + file)
        df.to_csv(tc_fast_steady + file.replace('.pkl', '.csv'), sep='\t', index=False)

for file in os.listdir(tc_fast_unsteady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(tc_fast_unsteady + file)
        df.to_csv(tc_fast_unsteady + file.replace('.pkl', '.csv'), sep='\t', index=False)

for file in os.listdir(of351_steady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(of351_steady + file)
        df.to_csv(of351_steady + file.replace('.pkl', '.csv'), sep='\t', index=False)

for file in os.listdir(of351_unsteady):
    if file.endswith('.out.pkl'):
        df = pd.read_pickle(of351_unsteady + file)
        df.to_csv(of351_unsteady + file.replace('.pkl', '.csv'), sep='\t', index=False)

In [ ]:
df[('Time', '(s)')][len(df[('Time', '(s)')]) - 1]

In [5]:
# process and save all the stats for all the folders

runFail = {'success':None, 'error':None, 'notes':None, 'author':'Chetan'}


# tc_steady
for file in os.listdir(tc_steady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in tc_steady')
        df = pd.read_pickle(tc_steady + file)
        stats, success = process_Stats(df)
        runFail['success'] = success
        with open(tc_steady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f: 
            pickle.dump(runFail, f)
        stats.to_pickle(tc_steady + file.replace('.out.pkl', '_stats.pkl'))

# tc_unsteady
for file in os.listdir(tc_unsteady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in tc_unsteady')
        df = pd.read_pickle(tc_unsteady + file)
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_unsteady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_pickle(tc_unsteady + file.replace('.out.pkl', '_stats.pkl'))

# tc_fast_steady
for file in os.listdir(tc_fast_steady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in tc_fast_steady')
        df = pd.read_pickle(tc_fast_steady + file)
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_fast_steady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_pickle(tc_fast_steady + file.replace('.out.pkl', '_stats.pkl'))

# tc_fast_unsteady
for file in os.listdir(tc_fast_unsteady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in tc_fast_unsteady')
        df = pd.read_pickle(tc_fast_unsteady + file)
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_fast_unsteady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_pickle(tc_fast_unsteady + file.replace('.out.pkl', '_stats.pkl'))

# of351_steady
for file in os.listdir(of351_steady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in of351_steady')
        df = pd.read_pickle(of351_steady + file)
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(of351_steady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_pickle(of351_steady + file.replace('.out.pkl', '_stats.pkl'))

# of351_unsteady
for file in os.listdir(of351_unsteady):
    if file.endswith('.out.pkl'):
        print(f'Processing {file[:-8]} in of351_unsteady')
        df = pd.read_pickle(of351_unsteady + file)
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(of351_unsteady + file.replace('.out.pkl', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_pickle(of351_unsteady + file.replace('.out.pkl', '_stats.pkl'))

Processing storm-test_09 in tc_steady
Processing storm-test_19 in tc_steady
Processing storm-test_26 in tc_steady
Processing storm-test_10 in tc_steady
Processing storm-test_00 in tc_steady
Processing storm-test_01 in tc_steady
Processing storm-test_11 in tc_steady
Processing storm-test_18 in tc_steady
Processing storm-test_08 in tc_steady
Processing storm-test_27 in tc_steady
Processing storm-test_13 in tc_steady
Processing storm-test_03 in tc_steady
Processing storm-test_25 in tc_steady
Processing storm-test_24 in tc_steady
Processing storm-test_02 in tc_steady
Processing storm-test_12 in tc_steady
Processing storm-test_28 in tc_steady
Processing storm-test_17 in tc_steady
Processing storm-test_07 in tc_steady
Processing storm-test_31 in tc_steady
Processing storm-test_21 in tc_steady
Processing storm-test_20 in tc_steady
Processing storm-test_30 in tc_steady
Processing storm-test_29 in tc_steady
Processing storm-test_06 in tc_steady
Processing storm-test_16 in tc_steady
Processing s

/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in sq

Processing storm-test_26 in tc_fast_steady
Processing storm-test_10 in tc_fast_steady
No data after 420.0 seconds
Processing storm-test_00 in tc_fast_steady
No data after 420.0 seconds
Processing storm-test_01 in tc_fast_steady
Processing storm-test_11 in tc_fast_steady
No data after 420.0 seconds


/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:290: RuntimeWarning: invalid value encountered in divide
  signal = signal / gain
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:291: RuntimeWarning: invalid value encountered in cast
  signal = np.round(signal).astype(int)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:691: RuntimeWarning: overflow encountered in scalar subtract
  if peak - trough >= R:
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDat

Processing storm-test_18 in tc_fast_steady
Processing storm-test_08 in tc_fast_steady
Processing storm-test_27 in tc_fast_steady
Processing storm-test_13 in tc_fast_steady
No data after 420.0 seconds
Processing storm-test_03 in tc_fast_steady
Processing storm-test_25 in tc_fast_steady
Processing storm-test_24 in tc_fast_steady
Processing storm-test_02 in tc_fast_steady
Processing storm-test_12 in tc_fast_steady
No data after 420.0 seconds


...

Processing storm-test_27 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_13 in tc_fast_unsteady
Processing storm-test_03 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_25 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_24 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_02 in tc_fast_unsteady
No data after 420.0 seconds


/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:98: RuntimeWarning: overflow encountered in power
  DELi = S**m * N / neq
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:98: RuntimeWarning: overflow encountered in power
  DELi = S**m * N / neq
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDatView/py

Processing storm-test_12 in tc_fast_unsteady
Processing storm-test_28 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_17 in tc_fast_unsteady
Processing storm-test_07 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_31 in tc_fast_unsteady
No data after 420.0 seconds


/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:290: RuntimeWarning: invalid value encountered in divide
  signal = signal / gain
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:291: RuntimeWarning: invalid value encountered in cast
  signal = np.round(signal).astype(int)
/Users/mchetan/Desktop/nrel/tools/pyDatView/pydatview-9aug22/pydatview/tools/fatigue.py:691: RuntimeWarning: overflow encountered in scalar subtract
  if peak - trough >= R:
/Users/mchetan/Desktop/nrel/projects/2-weis/env/weis-gui-dev/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/mchetan/Desktop/nrel/tools/pyDat

Processing storm-test_21 in tc_fast_unsteady
Processing storm-test_20 in tc_fast_unsteady
Processing storm-test_30 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_29 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_06 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_16 in tc_fast_unsteady
Processing storm-test_32 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_22 in tc_fast_unsteady
Processing storm-test_14 in tc_fast_unsteady
Processing storm-test_04 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_05 in tc_fast_unsteady
No data after 420.0 seconds
Processing storm-test_15 in tc_fast_unsteady
Processing storm-test_23 in tc_fast_unsteady
Processing storm-test_09 in of351_steady
Processing storm-test_19 in of351_steady
Processing storm-test_26 in of351_steady
No data after 420.0 seconds
Processing storm-test_10 in of351_steady
Processing storm-test_00 in of351_steady
Processing sto

In [ ]:

df1 = pd.read_pickle('eagleData/23JAN24/steady/storm-test_00.out.pkl')
stats, success = process_Stats(df1)         
stats.attribute = success
print(stats.attribute)

with open('testPickly.pkl', 'wb') as f:
    pickle.dump(stats, f)

with open('testPickly.pkl', 'rb') as f:
    df = pickle.load(f)

print(df.attribute)

df.attrs


In [48]:
# function to Create a Dataframe of the stats of all runs and plot it as a heatmap, NOT EFFICIENT BUT WORKS!

def pltstats(ChanName, statType):

    # dict of the paths
    runPaths = {'tc_steady': tc_steady, 'tc_unsteady': tc_unsteady, 'tc_fast_steady': tc_fast_steady, 'tc_fast_unsteady': tc_fast_unsteady, 'of351_steady': of351_steady, 'of351_unsteady': of351_unsteady}

    plotDF = pd.DataFrame(columns=runPaths.keys())

    # success matrix
    success = pd.DataFrame(columns=runPaths.keys())

    # looping through all the stat files then runt ypes


    # looping through cm_df rows
    for index, row in cm_df.iterrows():

        # looping through the run types dict
        for runType, path in runPaths.items():

            # building the file name
            fileName = f'{path}/storm-test_{index:02d}_stats.pkl'

            # reading the file
            # print(f'Processing {fileName}')
            df = pd.read_pickle(fileName)

            # adding the value to the plotDF
            plotDF.loc[index, runType] = df.loc[ChanName, statType]

            # adding the success flag to the success matrix
            with open(fileName.replace('_stats.pkl', '_runStatus.pkl'), 'rb') as f:
                runFail = pickle.load(f)
                success.loc[index, runType] = runFail['success']

    # plotting the heatmap
    plt.figure(figsize=(20, 20))
    ax = sns.heatmap(plotDF.apply(pd.to_numeric), annot=True, fmt=".2f", cmap='viridis')

    # # set failed runs to red text color
    # for i in range(success.shape[0]):
    #     for j in range(success.shape[1]):
    #         # print(f'failed run at {i}, {j} = {ax.texts[i*success.shape[0] + j]}')
    #         if success.iloc[i, j] == False:
    #             # setting the color of the cell text to red
                
    #             if ax.texts[i*success.shape[1] + j].get_text() != 'nan':
    #                 ax.texts[i*success.shape[1] + j].set_color('red')


    ax.set_title(f'{ChanName} {statType} for all runs')

    ax.xaxis.tick_top()

    # replace x ticks with yaw angles
    ax.set_yticks(ticks=np.array(cm_df.index), labels=np.array(cm_df[cm_df.columns[10]]).astype(float).round(0).astype(int))



    # Go through 

    ax.set_xlabel('Run Type')
    ax.set_ylabel('Yaw Angle [deg]')

    # plt.show()

    # Save the plot image in a new folder
    plotDF.to_pickle(f'HeatMaps/data/{ChanName}_{statType}.pkl') 

    plt.savefig(f'HeatMaps/{ChanName}_{statType}.png')

    plt.close()

    return success



In [ ]:
df = pd.read_pickle('eagleData/23JAN24/steady/storm-test_00.out.pkl')
canList = df.columns
list(canList)

In [49]:
InterstingChans = [
    ('TTDspFA', '(m)'),
    ('TTDspSS', '(m)'),
    ('TwrBsMxt', '(kN-m)'),
    ('TwrBsMyt', '(kN-m)'),
    
    ('B1RootMxr', '(N-m)'),
    ('B1RootMyr', '(N-m)'),
    ('B1TipTDxr', '(m)'),
    ('B1TipTDyr', '(m)'),

    ('B2RootMxr', '(N-m)'),
    ('B2RootMyr', '(N-m)'),
    ('B2TipTDxr', '(m)'),
    ('B2TipTDyr', '(m)'),

    ('B3RootMxr', '(N-m)'),
    ('B3RootMyr', '(N-m)'),
    ('B3TipTDxr', '(m)'),
    ('B3TipTDyr', '(m)'),

    ('B1N1Cl', '(-)'),
    ('B1N2Cl', '(-)'),
    ('B1N3Cl', '(-)'),
    ('B1N4Cl', '(-)'),
    ('B1N5Cl', '(-)'),
    ('B1N6Cl', '(-)'),
    ('B1N7Cl', '(-)'),
    ('B1N8Cl', '(-)'),
    ('B1N9Cl', '(-)'),
]

# Single test run
status = pltstats(('B1N9Cl', '(-)'), 'mean')

# for chan in InterstingChans:
#     pltstats(chan, 'mean')
#     pltstats(chan, 'std')
#     pltstats(chan, 'DEL5')
#     pltstats(chan, 'DEL10')
#     pltstats(chan, 'DEL12')

In [54]:
status

,tc_steady,tc_unsteady,tc_fast_steady,tc_fast_unsteady,of351_steady,of351_unsteady
0,True,False,False,False,True,False
1,True,False,True,False,True,False
2,True,False,True,False,True,False
3,True,False,True,False,True,False
4,True,False,True,False,True,False
5,True,False,True,False,True,False
6,True,False,True,False,True,False
7,True,False,True,False,True,False
8,True,False,True,False,False,False
9,True,True,True,True,True,False
